EJEMPLO BÁSICO INSTALACIÓN

In [12]:
! conda install typing-extensions=3.10


Channels:
 - defaults
 - conda-forge
 - rapidsai
 - nvidia
 - fastai
 - pytorch
 - timeseriesai
Platform: linux-64
Solving environment: failed

LibMambaUnsatisfiableError: Encountered problems while solving:
  - package typing-extensions-3.10.0.0-hd3eb1b0_0 requires typing_extensions 3.10.0.0 pyh06a4308_0, but none of the providers can be installed

Could not solve for environment specs
The following packages are incompatible
├─ pin-1 is installable and it requires
│  └─ python 3.10.* , which can be installed;
├─ typing-extensions 3.10**  is installable with the potential options
│  ├─ typing-extensions 3.10.0.0 would require
│  │  └─ typing_extensions 3.10.0.0 pyh06a4308_0, which can be installed;
│  ├─ typing-extensions 3.10.0.2 would require
│  │  └─ typing_extensions 3.10.0.2 pyh06a4308_0, which can be installed;
│  ├─ typing-extensions 3.10.0.0 would require
│  │  └─ typing_extensions 3.10.0.0 pyha770c72_0, which can be installed;
│  ├─ typing-extensions 3.10.0.1 would require
│  

In [10]:
! conda install anaconda::gluonts

Channels:
 - defaults
 - anaconda
 - conda-forge
 - rapidsai
 - nvidia
 - fastai
 - pytorch
 - timeseriesai
Platform: linux-64
Solving environment: / warning  libmamba Added empty dependency for problem type SOLVER_RULE_UPDATE
failed

LibMambaUnsatisfiableError: Encountered problems while solving:
  - package gluonts-0.8.1-pyhd3eb1b0_0 requires typing-extensions ~=3.10.0.0, but none of the providers can be installed

Could not solve for environment specs
The following packages are incompatible
├─ gluonts is installable with the potential options
│  ├─ gluonts 0.8.1 would require
│  │  └─ typing-extensions ~=3.10.0.0  with the potential options
│  │     ├─ typing-extensions 3.10.0.0 would require
│  │     │  └─ typing_extensions 3.10.0.0 pyh06a4308_0, which can be installed;
│  │     ├─ typing-extensions 3.10.0.2 would require
│  │     │  └─ typing_extensions 3.10.0.2 pyh06a4308_0, which can be installed;
│  │     ├─ typing-extensions 3.10.0.0 would require
│  │     │  └─ typing_extensi

In [11]:
import torch
import matplotlib.pyplot as plt
import pandas as pd
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import split
from huggingface_hub import hf_hub_download

from uni2ts.eval_util.plot import plot_single
from uni2ts.model.moirai import MoiraiForecast, MoiraiModule

ModuleNotFoundError: No module named 'gluonts'

In [ ]:
SIZE = "small"  # model size: choose from {'small', 'base', 'large'}
PDT = 20  # prediction length: any positive integer
CTX = 200  # context length: any positive integer
PSZ = "auto"  # patch size: choose from {"auto", 8, 16, 32, 64, 128}
BSZ = 32  # batch size: any positive integer
TEST = 100  # test set length: any positive integer



In [ ]:
# Read data into pandas DataFrame
url = (
    "https://gist.githubusercontent.com/rsnirwan/c8c8654a98350fadd229b00167174ec4"
    "/raw/a42101c7786d4bc7695228a0f2c8cea41340e18f/ts_wide.csv"
)

In [ ]:
df = pd.read_csv(url, index_col=0, parse_dates=True)

In [ ]:
# Convert into GluonTS dataset
ds = PandasDataset(dict(df))

In [ ]:
# Split into train/test set
train, test_template = split(
    ds, offset=-TEST
)  # assign last TEST time steps as test set

In [ ]:
# Construct rolling window evaluation
test_data = test_template.generate_instances(
    prediction_length=PDT,  # number of time steps for each prediction
    windows=TEST // PDT,  # number of windows in rolling window evaluation
    distance=PDT,  # number of time steps between each window - distance=PDT for non-overlapping windows
)

In [ ]:
# Prepare pre-trained model by downloading model weights from huggingface hub
model = MoiraiForecast(
    module=MoiraiModule.from_pretrained(f"Salesforce/moirai-1.0-R-{SIZE}"),
    prediction_length=PDT,
    context_length=CTX,
    patch_size=PSZ,
    num_samples=100,
    target_dim=1,
    feat_dynamic_real_dim=ds.num_feat_dynamic_real,
    past_feat_dynamic_real_dim=ds.num_past_feat_dynamic_real,
)

In [ ]:
predictor = model.create_predictor(batch_size=BSZ)
forecasts = predictor.predict(test_data.input)

In [ ]:
input_it = iter(test_data.input)
label_it = iter(test_data.label)
forecast_it = iter(forecasts)

In [ ]:
inp = next(input_it)
label = next(label_it)
forecast = next(forecast_it)

In [ ]:
plot_single(
    inp, 
    label, 
    forecast, 
    context_length=200,
    name="pred",
    show_label=True,
)

In [ ]:
plt.show()